In [1]:
def load_my_modules(prj_name=None, lst=None):

    import sys, os

    if prj_name==None:  prj_name = 'project_KnowledgeBase'       
    if lst==None:       lst = [
            '\\my_methods',
            '\\personal_expenses\\modules'
            # '\\qqq,',        
    ]

    # Получить текущую папку
    cwd = os.getcwd()
    prj_len = len(prj_name)
    i1 = cwd.find(prj_name)
    prj_dir = cwd[0:i1+prj_len]
    # d:\IDE_prj\project_KnowledgeBase

    # '\' -> '\\'
    if not prj_dir.find('\\\\'):
        prj_dir = prj_dir.replace('\\', '\\\\')
    
    # Проверить наличие модулей в sys.path_hook
    # Если отсутствют - добавить
    for module_dir in lst:
        module_dir = prj_dir + module_dir
        if not module_dir in sys.path:
            sys.path.insert(0, module_dir)
    return sys.path
load_my_modules()

['d:\\IDE_prj\\project_KnowledgeBase\\personal_expenses\\modules',
 'd:\\IDE_prj\\project_KnowledgeBase\\my_methods',
 'd:\\IDE_prj\\project_KnowledgeBase\\personal_expenses',
 'C:\\Users\\Александр\\AppData\\Local\\Programs\\Python\\Python39\\python39.zip',
 'C:\\Users\\Александр\\AppData\\Local\\Programs\\Python\\Python39\\DLLs',
 'C:\\Users\\Александр\\AppData\\Local\\Programs\\Python\\Python39\\lib',
 'C:\\Users\\Александр\\AppData\\Local\\Programs\\Python\\Python39',
 '',
 'C:\\Users\\Александр\\AppData\\Roaming\\Python\\Python39\\site-packages',
 'C:\\Users\\Александр\\AppData\\Roaming\\Python\\Python39\\site-packages\\win32',
 'C:\\Users\\Александр\\AppData\\Roaming\\Python\\Python39\\site-packages\\win32\\lib',
 'C:\\Users\\Александр\\AppData\\Roaming\\Python\\Python39\\site-packages\\Pythonwin',
 'C:\\Users\\Александр\\AppData\\Local\\Programs\\Python\\Python39\\lib\\site-packages']

In [118]:
import os
path = 'data_input/my_tickets_json'
lst_files = os.listdir(path)
for fn in lst_files:
    save_json_to_database(fn)
    break

In [8]:
import json
import pandas as pd
import datetime as dt
from models import *
import os


def save_json_to_database(file_name):

    try:
        with open(file_name, encoding='utf-8') as json_file:
            data = json.load(json_file) #, cls= encoding='cp1251').
            head=data[0]
            items = head.pop('items')
            df=pd.DataFrame(items)

            # Запись в БД. Транзакции удаления/записи    
            with db.atomic() as transaction:  # transaction.rollback()    
                try:
                    # записать чек (заголовок)                    
                    # columns=["date", "date_account", "category", "name", "amount", "balance"]
                    rec = head
                    receipt = FNS_Receipt(
                            dateTime              = dt.datetime.fromtimestamp(rec['dateTime']),
                            localDateTime         = dt.datetime.strptime(rec['localDateTime'], '%Y-%m-%dT%H:%M'),
                            totalSum              = rec['totalSum']/100,
                            user                  = rec['user'],
                            retailPlaceAddress    = rec['retailPlaceAddress'],
                            userInn               = rec['userInn'],
                            requestNumber         = rec['requestNumber'],
                            shiftNumber           = rec['shiftNumber'],
                            operator              = rec['operator'],
                            operationType         = rec['operationType'],
                            cashTotalSum          = rec['cashTotalSum'],
                            ecashTotalSum         = rec['ecashTotalSum'],
                            kktRegId              = rec['kktRegId'],
                            fiscalDriveNumber     = rec['fiscalDriveNumber'],
                            fiscalDocumentNumber  = rec['fiscalDocumentNumber'],
                            fiscalSign            = rec['fiscalSign'],
                            nds18                 = '', #rec['nds18'],
                            nds10                 = '', #rec['nds10'],
                            code                  = rec['code'],
                            fiscalDocumentFormatVer = rec['fiscalDocumentFormatVer'],
                            retailPlace             = rec['retailPlace'],
                            prepaidSum              = rec['prepaidSum'],
                            creditSum               = rec['creditSum'],
                            provisionSum            = rec['provisionSum'],
                            taxationType            = rec['taxationType'],
                    )
                    receipt.save()

                    # Записать номенклатуру покупок
                    lst = []
                    for i,rec in df.iterrows():
                        obj = FNS_Receipt_Product(
                            receipt_id   = receipt.id,
                            name         = rec['name'],        # object 
                            price        = rec['price']/100,   # int64  
                            amount       = rec['sum']/100,     # int64  
                            quantity     = rec['quantity'],    # float64
                            paymentType  = rec['paymentType'], # int64  
                            productType  = rec['productType'], # int64  
                            nds          = '' #rec['nds'],         # int64  
                        )
                        lst.append(obj)
                    FNS_Receipt_Product.bulk_create(lst, batch_size=999)
                    
                except Exception as e:                    
                    print('При попытке сохранить в базу данных загруженные данные возникла ошибка')
                    print(e)
                    transaction.rollback()
                    raise
    except:
        return file_name + ' - fail'        
    return file_name + ' - Ok'      

# path = 'data_input/my_tickets_json'
# lst_files = os.listdir(path)
# for fn in lst_files:
#     res = save_json_to_database(path + '/' + fn)
#     print(res)

save_json_to_database('data_input/my_tickets_json/20_04_2022_01_31_397657834228145796699.json')


При попытке сохранить в базу данных загруженные данные возникла ошибка
'nds18'


'data_input/my_tickets_json/20_04_2022_01_31_397657834228145796699.json - fail'

KeyboardInterrupt: 

In [6]:
db.close()

False

In [69]:
import json

# file_name = 'data_input/tests/test_01.json'
file_name = 'data_input/tests/test_02_main.json'
data=None

with open(file_name, encoding='utf-8') as json_file:
    data = json.load(json_file) #, cls= encoding='cp1251').

head=data[0]
items = head.pop('items')

# for ind in head:
#     print(f'{ind}: ',  head[ind])
# print('------------')
# print(items)
# for ind in items:
#     print(ind)
